In [13]:
import findspark
findspark.init()
# Spark:
from pyspark     import SparkConf
from pyspark     import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
# Python :
from datetime import datetime
import math
from pyspark.streaming import StreamingContext

listen_to_ip   = "54.37.12.49"
listen_to_port = 12012

current_date   = datetime.now().strftime("%Y-%m-%d %H:%M")
user           = "romain - exemple saprk streaming"
appName        = "%s le %s"%(user, current_date)
ip_fares       = "54.37.12.49"
master         = "spark://54.37.12.49:7077"
executor_cores = 2 
nb_cores_max   = 10
parallelism    = 4 * executor_cores
memory_by_node = "200M"
nb_executor    = nb_cores_max//executor_cores
dico_conf      = { "spark.app.name"             : appName        ,
                   "spark.master"               : master         ,
                   "spark.cores.max"            : nb_cores_max   , 
                   "spark.executor.cores"       : executor_cores ,
                   "spark.default.parallelism"  : parallelism    , 
                  #"spark.python.worker.memory" : memory_by_node ,
                   "spark.python.executor.memory" : memory_by_node }

conf = SparkConf()
for k,v in dico_conf.items():
    conf = conf.set(k,v)
sc = SparkContext.getOrCreate(conf=conf)

batch_duration = 1
ssc         = StreamingContext(sc, bacth_duration)
dstream     = ssc.socketTextStream(listen_to_ip, listen_to_port)

In [14]:
ssc.checkpoint("hdfs://54.37.12.49:8020/spark_streaming_checkpoint/")

In [15]:
def update_achats(nouvelles_valeurs, valeur_actuelle ):
    if valeur_actuelle is None:
        valeur_actuelle = 0
    return sum(nouvelles_valeurs, valeur_actuelle)
        

In [16]:
data            = dstream.map(lambda x: x.split(","))
clients_facture = data.map(lambda x: (x[0], float(x[1])*float(x[2])))


In [17]:
# on veut savoir toutes les 5 secondes, ce qu'ils ont acheté pendant les 10 dernières secondes
update_client   = clients_facture.reduceByKeyAndWindow(lambda x,y : x+y, 
                                                       lambda x,y : x+y, 
                                                       windowDuration = batch_duration*10, 
                                                       slideDuration  = batch_duration*5 )
update_client.pprint()

# 9) activer l'écoute sur la socket

In [18]:
ssc.start()

-------------------------------------------
Time: 2018-02-26 14:58:01
-------------------------------------------
('client_c', 1139.0)
('client_b', 1410.0)
('client_a', 743.0)
('client_d', 1049.0)

-------------------------------------------
Time: 2018-02-26 14:58:06
-------------------------------------------
('client_c', 2196.0)
('client_b', 2932.0)
('client_a', 2801.0)
('client_d', 3215.0)

-------------------------------------------
Time: 2018-02-26 14:58:11
-------------------------------------------
('client_c', 3287.0)
('client_b', 3291.0)
('client_a', 2693.0)
('client_d', 3382.0)

-------------------------------------------
Time: 2018-02-26 14:58:16
-------------------------------------------
('client_c', 3007.0)
('client_b', 3653.0)
('client_a', 3034.0)
('client_d', 2852.0)

-------------------------------------------
Time: 2018-02-26 14:58:21
-------------------------------------------
('client_c', 2084.0)
('client_b', 3358.0)
('client_a', 4568.0)
('client_d', 2714.0)

------

In [19]:
ssc.stop(False, True)

-------------------------------------------
Time: 2018-02-26 14:59:11
-------------------------------------------

